In [17]:
user = "Alex"
pathtorepo = "C:\\Users\\" *user *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\bifurcation\\")

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\bifurcation`


In [18]:
using BifurcationKit, LinearAlgebra, Plots, Parameters, JLD2
const BK = BifurcationKit

BifurcationKit

In [20]:
τ_ = 0.013; τD_ = 0.07993;  τy_ = 3.3; J_ = 3.07; β_ = 0.300
xthr_ = 0.75; ythr_ = 0.4
α_ = 1.58; ΔU0_ = 0.305
U0_, I0_ = 0.25, -1.6

@inbounds function TM_bk!(du, u, p, t = 0)
    U_ = p[8] + p[9] / ( 1.0 + exp( -50.0 * (u[3] - p[7]) ) )
    σ = 1.0 / ( 1.0 + exp( -20.0 * (u[2]-p[6]) ) )
    g(E, x, y, p, U_) = log( 1.0 + exp( (p[5] * U_ * x * E + p[11]  ) / (p[1]) ) )
    
    du[1] = (-u[1] + p[1] * g(u[1], u[2], u[3], p, U_) ) / p[2]
    du[2] = (1.0 - u[2]) / p[3] - U_*u[2]*u[1]
    du[3] = (-u[3])/p[4] + p[10] * σ
    
    return du
end

TM_bk! (generic function with 2 methods)

In [24]:
opt_new = NewtonPar(maxIter = 15, tol = 1e-6)
pmax, pmin = 0.0, -1.74

opts_con = ContinuationPar(pMin = pmin, pMax = pmax,
                            ds = 0.001, dsmin = 1e-5, dsmax = 0.1,
                            nev = 3, detectBifurcation = 3, newtonOptions  = opt_new,
                            maxSteps  = 0, nInversion = 6);

In [29]:
TM_bk(u, p) = TM_bk!(similar(u), u, p, 0)
G(u, p) = u .- TM_bk(u, p)
p_tm = (α = α_, τ = τ_, τD = τD_, τy = τy_, J = J_, xthr = xthr_, ythr = ythr_, U0 = U0_, ΔU0 = ΔU0_, β = β_, I0 = I0_)
fp0 = [ 4.34581,  0.738495,  0.438299]
prob =  BifurcationProblem(G, fp0, p_tm, (@lens _.I0))
br = continuation(prob, PALC(), opts_con);

In [30]:
br.sol

1-element Vector{NamedTuple{(:x, :p, :step), Tuple{Vector{Float64}, Float64, Int64}}}:
 (x = [0.6071505915869747, 0.9156968958308669, 0.22215228872639023], p = -1.6, step = 0)

In [28]:
TM_bk([ 0.6071502354571418,
0.9156968943458036,
0.22215228872714327], p_tm)

3-element Vector{Float64}:
 0.6071717615601017
 0.9156969961822814
 0.22215228842299